In [2]:
import sys

In [3]:
sys.path.append('..')

In [28]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from collections import Counter
import spacy
import re
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spellchecker import SpellChecker
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
import spacy
import re
import lightgbm as lgb
from transformers import AutoTokenizer
from sklearn.metrics import mean_squared_error
import copy

In [36]:
import warnings

In [37]:
warnings.filterwarnings('ignore')

In [22]:
import random

In [23]:
import optuna
random.seed(42)
np.random.seed(42)
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [5]:
from cles.lgb_utils import Preprocessor

In [13]:
oof_attn_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_attn_pool.csv')
oof_conc_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_conc_pool.csv')
oof_gem_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_gem_pool.csv')
oof_lstm_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_lstm_pool.csv')
oof_mean_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_mean_pool.csv')
oof_gru_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_gru_pool.csv')
oof_meanmax_pool = pd.read_csv('../oof_predictions/microsoft_deberta_v3_large_meanmax_pool.csv')

In [14]:
# 'content_mean_pool', 'wording_mean_pool',
#        'content_lstm_pool', 'wording_lstm_pool', 'content_attn_pool',
#        'wording_attn_pool', 'content_gem_pool', 'wording_gem_pool',
#        'content_conc_pool', 'wording_conc_pool'

In [15]:
oof_attn_pool = oof_attn_pool.rename({
    'pred_content': 'content_attn_pool', 
    'pred_wording': 'wording_attn_pool'}, axis=1).drop(['content', 'wording'], axis=1)
oof_conc_pool = oof_conc_pool.rename({
    'pred_content': 'content_conc_pool',
    'pred_wording': 'wording_conc_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

oof_gem_pool = oof_gem_pool.rename({
    'pred_content': 'content_gem_pool',
    'pred_wording': 'wording_gem_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

oof_lstm_pool = oof_lstm_pool.rename({
    'pred_content': 'content_lstm_pool',
    'pred_wording': 'wording_lstm_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

oof_mean_pool = oof_mean_pool.rename({
    'pred_content': 'content_mean_pool',
    'pred_wording': 'wording_mean_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

oof_gru_pool = oof_gru_pool.rename({
    'pred_content': 'content_gru_pool',
    'pred_wording': 'wording_gru_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

oof_meanmax_pool = oof_meanmax_pool.rename({
    'pred_content': 'content_meanmax_pool',
    'pred_wording': 'wording_meanmax_pool'
}, axis=1).drop(['content', 'wording'], axis=1)

In [16]:
oof_df =oof_attn_pool.merge(
    oof_conc_pool, on=['student_id', 'prompt_id']
).merge(oof_gem_pool, on=['student_id', 'prompt_id']).merge(
    oof_lstm_pool, on=['student_id', 'prompt_id']
).merge(oof_mean_pool, on=['student_id', 'prompt_id']).merge(
    oof_gru_pool, on=['student_id', 'prompt_id']).merge(
    oof_meanmax_pool, on=['student_id', 'prompt_id'])

In [17]:
oof_df.columns

Index(['prompt_id', 'student_id', 'content_attn_pool', 'wording_attn_pool',
       'content_conc_pool', 'wording_conc_pool', 'content_gem_pool',
       'wording_gem_pool', 'content_lstm_pool', 'wording_lstm_pool',
       'content_mean_pool', 'wording_mean_pool', 'content_gru_pool',
       'wording_gru_pool', 'content_meanmax_pool', 'wording_meanmax_pool'],
      dtype='object')

In [18]:
preprocessor = Preprocessor()

In [19]:
prompts = pd.read_csv('../data_raw/prompts_train.csv')
summaries = pd.read_csv('../data_raw/summaries_train.csv')

In [20]:
df = preprocessor.run(prompts, summaries)

In [24]:
df.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'summary_length', 'splling_err_num', 'prompt_question', 'prompt_title',
       'prompt_text', 'prompt_length', 'length_ratio', 'word_overlap_count',
       'word_overlap_ratio', 'bigram_overlap_count', 'bigram_overlap_ratio',
       'trigram_overlap_count', 'trigram_overlap_ratio', 'quotes_count',
       'number_of_unique_words', 'flesch_kinsaid', 'flesch', 'gunning_fog',
       'coleman_liau', 'dale_chall', 'ari', 'linsear_write', 'smog', 'spache'],
      dtype='object')

In [12]:
#oof_df = oof_df.drop(['content', 'wording'], axis=1)

In [25]:
full_df = df.merge(oof_df, on = ['student_id', 'prompt_id'])

In [26]:
feature_list = full_df.drop(['student_id', 'prompt_id', 'text', 'content', 'wording',
                                       'prompt_question', 'prompt_title',
                                       'prompt_text'
                            ], axis=1).columns.tolist()

In [27]:
feature_list

['summary_length',
 'splling_err_num',
 'prompt_length',
 'length_ratio',
 'word_overlap_count',
 'word_overlap_ratio',
 'bigram_overlap_count',
 'bigram_overlap_ratio',
 'trigram_overlap_count',
 'trigram_overlap_ratio',
 'quotes_count',
 'number_of_unique_words',
 'flesch_kinsaid',
 'flesch',
 'gunning_fog',
 'coleman_liau',
 'dale_chall',
 'ari',
 'linsear_write',
 'smog',
 'spache',
 'content_attn_pool',
 'wording_attn_pool',
 'content_conc_pool',
 'wording_conc_pool',
 'content_gem_pool',
 'wording_gem_pool',
 'content_lstm_pool',
 'wording_lstm_pool',
 'content_mean_pool',
 'wording_mean_pool',
 'content_gru_pool',
 'wording_gru_pool',
 'content_meanmax_pool',
 'wording_meanmax_pool']

In [29]:
FEATURE_LIST_CONTENT = copy.deepcopy(feature_list)

In [33]:
def lgb_objective(trial):
    
    max_depth = trial.suggest_int('max_depth', 2, 10)
    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': max_depth,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2**max_depth - 1),
        'verbosity': -1  # Add this line to suppress warnings and info messages

    }
    
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[FEATURE_LIST_CONTENT], label=x_train['content'])
        dtest = lgb.Dataset(data=data[FEATURE_LIST_CONTENT], label=data['content'])
        
        model = lgb.train(params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[FEATURE_LIST_CONTENT])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['content', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['content'], res_df['prediction'], squared=False)
    return score
    

In [38]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))

In [39]:
study.optimize(lgb_objective, n_trials=1000)

In [40]:
study.best_params

{'max_depth': 2,
 'learning_rate': 0.06217180233323965,
 'lambda_l1': 3.809719942414819,
 'lambda_l2': 0.0001136302094037265,
 'num_leaves': 3}

In [41]:
study.best_value

0.42001436985625956

In [42]:
lgb_content_params = {
    'boosting_type': 'gbdt',
    'random_state': 42,
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.06217180233323965,
    'max_depth': 2,
    'lambda_l1': 3.809719942414819,
    'lambda_l2': 0.0001136302094037265,
    'num_leaves': 3,
    'verbosity': -1  # Add this line to suppress warnings and info messages
}

In [49]:
def feature_selection(trial):
    # 'summary_length', 'splling_err_num',
    #    'prompt_length', 'length_ratio', 'word_overlap_count',
    #    'bigram_overlap_count', 'bigram_overlap_ratio', 'trigram_overlap_count',
    #    'trigram_overlap_ratio', 'quotes_count', 'number_of_unique_words',
    #    'jaccard_similarity', 'pred_content', 'pred_wording'
    feature_dict = {}
    for feature in FEATURE_LIST_CONTENT:
        if 'content' in feature:
            feature_dict[feature] = 'use'
        else:
            feature_dict[feature] = trial.suggest_categorical(feature, ('use', 'drop'))
        
    
    new_feature_list = [key for key, value in feature_dict.items() if value != 'drop']
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[new_feature_list], label=x_train['content'])
        dtest = lgb.Dataset(data=data[new_feature_list], label=data['content'])
        
        model = lgb.train(lgb_content_params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[new_feature_list])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['content', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['content'], res_df['prediction'], squared=False)
    return score
    

In [50]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))

In [51]:
study.optimize(feature_selection, n_trials=1000)

In [52]:
feature_dict = study.best_params

In [53]:
feature_dict

{'summary_length': 'drop',
 'splling_err_num': 'drop',
 'prompt_length': 'drop',
 'length_ratio': 'drop',
 'word_overlap_count': 'drop',
 'word_overlap_ratio': 'drop',
 'bigram_overlap_count': 'drop',
 'bigram_overlap_ratio': 'use',
 'trigram_overlap_count': 'drop',
 'trigram_overlap_ratio': 'use',
 'quotes_count': 'drop',
 'number_of_unique_words': 'drop',
 'flesch_kinsaid': 'use',
 'flesch': 'drop',
 'gunning_fog': 'use',
 'coleman_liau': 'drop',
 'dale_chall': 'drop',
 'ari': 'drop',
 'linsear_write': 'use',
 'smog': 'drop',
 'spache': 'drop',
 'wording_attn_pool': 'use',
 'wording_conc_pool': 'drop',
 'wording_gem_pool': 'drop',
 'wording_lstm_pool': 'drop',
 'wording_mean_pool': 'use',
 'wording_gru_pool': 'use',
 'wording_meanmax_pool': 'drop'}

In [54]:
study.best_value

0.41659761796847694

In [55]:
UPDATED_FEATURE_LIST_CONTENT = []
for feature in FEATURE_LIST_CONTENT:
    if feature not in feature_dict.keys():
        UPDATED_FEATURE_LIST_CONTENT.append(feature)
    else:
        if feature_dict[feature] == 'use':
            UPDATED_FEATURE_LIST_CONTENT.append(feature)

In [56]:
UPDATED_FEATURE_LIST_CONTENT

['bigram_overlap_ratio',
 'trigram_overlap_ratio',
 'flesch_kinsaid',
 'gunning_fog',
 'linsear_write',
 'content_attn_pool',
 'wording_attn_pool',
 'content_conc_pool',
 'content_gem_pool',
 'content_lstm_pool',
 'content_mean_pool',
 'wording_mean_pool',
 'content_gru_pool',
 'wording_gru_pool',
 'content_meanmax_pool']

In [58]:
def lgb_objective(trial):
    
    max_depth = trial.suggest_int('max_depth', 2, 10)
    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': max_depth,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2**max_depth - 1),
        'verbosity': -1  # Add this line to suppress warnings and info messages

    }
    
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[UPDATED_FEATURE_LIST_CONTENT], label=x_train['content'])
        dtest = lgb.Dataset(data=data[UPDATED_FEATURE_LIST_CONTENT], label=data['content'])
        
        model = lgb.train(params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[UPDATED_FEATURE_LIST_CONTENT])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['content', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['content'], res_df['prediction'], squared=False)
    return score

In [59]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(lgb_objective, n_trials=1000)

In [60]:
study.best_params

{'max_depth': 3,
 'learning_rate': 0.036896529166717484,
 'lambda_l1': 1.1243982150236251,
 'lambda_l2': 0.06777200422733183,
 'num_leaves': 3}

In [61]:
content_best_score = study.best_value

In [62]:
content_best_score

0.4154742412317312

In [63]:
best_params_content = {
    'boosting_type': 'gbdt',
    'random_state': 42,
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.036896529166717484,
    'max_depth': 3,
    'lambda_l1': 1.1243982150236251,
    'lambda_l2': 0.06777200422733183,
    'num_leaves': 3,
    'verbosity': -1  # Add this line to suppress warnings and info messages
}

In [64]:
UPDATED_FEATURE_LIST_CONTENT

['bigram_overlap_ratio',
 'trigram_overlap_ratio',
 'flesch_kinsaid',
 'gunning_fog',
 'linsear_write',
 'content_attn_pool',
 'wording_attn_pool',
 'content_conc_pool',
 'content_gem_pool',
 'content_lstm_pool',
 'content_mean_pool',
 'wording_mean_pool',
 'content_gru_pool',
 'wording_gru_pool',
 'content_meanmax_pool']

In [65]:
FEATURE_LIST_WORDING = copy.deepcopy(feature_list)

In [80]:
FEATURE_LIST_WORDING

['summary_length',
 'splling_err_num',
 'prompt_length',
 'length_ratio',
 'word_overlap_count',
 'word_overlap_ratio',
 'bigram_overlap_count',
 'bigram_overlap_ratio',
 'trigram_overlap_count',
 'trigram_overlap_ratio',
 'quotes_count',
 'number_of_unique_words',
 'flesch_kinsaid',
 'flesch',
 'gunning_fog',
 'coleman_liau',
 'dale_chall',
 'ari',
 'linsear_write',
 'smog',
 'spache',
 'content_attn_pool',
 'wording_attn_pool',
 'content_conc_pool',
 'wording_conc_pool',
 'content_gem_pool',
 'wording_gem_pool',
 'content_lstm_pool',
 'wording_lstm_pool',
 'content_mean_pool',
 'wording_mean_pool',
 'content_gru_pool',
 'wording_gru_pool',
 'content_meanmax_pool',
 'wording_meanmax_pool']

In [68]:
def lgb_objective(trial):
    
    max_depth = trial.suggest_int('max_depth', 2, 10)
    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': max_depth,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2**max_depth - 1),
        'verbosity': -1  # Add this line to suppress warnings and info messages

    }
    
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[FEATURE_LIST_WORDING], label=x_train['wording'])
        dtest = lgb.Dataset(data=data[FEATURE_LIST_WORDING], label=data['wording'])
        
        model = lgb.train(params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[FEATURE_LIST_WORDING])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['wording', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['wording'], res_df['prediction'], squared=False)
    return score

In [69]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(lgb_objective, n_trials=1000)

In [70]:
study.best_params

{'max_depth': 3,
 'learning_rate': 0.05925967774388273,
 'lambda_l1': 0.8205074986845817,
 'lambda_l2': 1.2150694364937778e-06,
 'num_leaves': 4}

In [71]:
study.best_value

0.5651926861202722

In [72]:
lgb_wording_params = {
    'boosting_type': 'gbdt',
    'random_state': 42,
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.05925967774388273,
    'max_depth': 3,
    'lambda_l1': 0.8205074986845817,
    'lambda_l2': 1.2150694364937778e-06,
    'num_leaves': 4,
    'verbosity': -1  # Add this line to suppress warnings and info messages
}

In [73]:
def feature_selection(trial):
    # 'summary_length', 'splling_err_num',
    #    'prompt_length', 'length_ratio', 'word_overlap_count',
    #    'bigram_overlap_count', 'bigram_overlap_ratio', 'trigram_overlap_count',
    #    'trigram_overlap_ratio', 'quotes_count', 'number_of_unique_words',
    #    'jaccard_similarity', 'pred_content', 'pred_wording'
    feature_dict = {}
    for feature in FEATURE_LIST_WORDING:
        if 'wording' in feature:
            feature_dict[feature] = 'use'
        else:
            feature_dict[feature] = trial.suggest_categorical(feature, ('use', 'drop'))
    
    new_feature_list = [key for key, value in feature_dict.items() if value != 'drop']
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[new_feature_list], label=x_train['wording'])
        dtest = lgb.Dataset(data=data[new_feature_list], label=data['wording'])
        
        model = lgb.train(lgb_wording_params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[new_feature_list])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['wording', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['wording'], res_df['prediction'], squared=False)
    return score

In [74]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(feature_selection, n_trials=1000)

In [81]:
feature_dict = study.best_params

In [82]:
UPDATED_FEATURE_LIST_WORDING = []
for feature in FEATURE_LIST_WORDING:
    if feature not in feature_dict.keys():
        UPDATED_FEATURE_LIST_WORDING.append(feature)
    else:
        if feature_dict[feature] == 'use':
            UPDATED_FEATURE_LIST_WORDING.append(feature)

In [83]:
UPDATED_FEATURE_LIST_WORDING

['length_ratio',
 'word_overlap_count',
 'word_overlap_ratio',
 'bigram_overlap_count',
 'bigram_overlap_ratio',
 'quotes_count',
 'linsear_write',
 'wording_attn_pool',
 'content_conc_pool',
 'wording_conc_pool',
 'content_gem_pool',
 'wording_gem_pool',
 'wording_lstm_pool',
 'wording_mean_pool',
 'wording_gru_pool',
 'wording_meanmax_pool']

In [84]:
study.best_value

0.5655373600157818

In [85]:
def lgb_objective(trial):
    
    max_depth = trial.suggest_int('max_depth', 2, 10)
    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': max_depth,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2**max_depth - 1),
        'verbosity': -1  # Add this line to suppress warnings and info messages

    }
    
    res = []
    for prompt_id, data in full_df.groupby('prompt_id'):
        
        x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
        dtrain = lgb.Dataset(data=x_train[UPDATED_FEATURE_LIST_WORDING], label=x_train['wording'])
        dtest = lgb.Dataset(data=data[UPDATED_FEATURE_LIST_WORDING], label=data['wording'])
        
        model = lgb.train(params,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=False)])
        
        predictions = model.predict(data[UPDATED_FEATURE_LIST_WORDING])
        data.loc[:, 'prediction'] = predictions
        res.append(data[['wording', 'prediction']].copy())
        
    res_df = pd.concat(res, axis=0, ignore_index=True)
    score = mean_squared_error(res_df['wording'], res_df['prediction'], squared=False)
    return score

In [86]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(lgb_objective, n_trials=1000)

In [87]:
best_score_wording = study.best_value

In [88]:
best_score_wording

0.5618506924647094

In [89]:
study.best_params

{'max_depth': 5,
 'learning_rate': 0.062143850305355686,
 'lambda_l1': 0.008924228670105123,
 'lambda_l2': 0.238561343589459,
 'num_leaves': 8}

In [90]:
best_params_wording = {
    'boosting_type': 'gbdt',
    'random_state': 42,
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.062143850305355686,
    'max_depth': 5,
    'lambda_l1': 0.008924228670105123,
    'lambda_l2': 0.238561343589459,
    'num_leaves': 8,
    'verbosity': -1  # Add this line to suppress warnings and info messages
}

In [91]:
UPDATED_FEATURE_LIST_WORDING

['length_ratio',
 'word_overlap_count',
 'word_overlap_ratio',
 'bigram_overlap_count',
 'bigram_overlap_ratio',
 'quotes_count',
 'linsear_write',
 'wording_attn_pool',
 'content_conc_pool',
 'wording_conc_pool',
 'content_gem_pool',
 'wording_gem_pool',
 'wording_lstm_pool',
 'wording_mean_pool',
 'wording_gru_pool',
 'wording_meanmax_pool']

In [92]:
#MCRMSE
(best_score_wording + content_best_score)/2

0.48866246684822034

In [93]:
## create oof predictions

In [94]:
full_df.loc[:, 'pred_lgb_content'] = None
full_df.loc[:, 'pred_lgb_wording'] = None

In [96]:
for prompt_id, data in full_df.groupby('prompt_id'):
    x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
    dtrain = lgb.Dataset(data=x_train[UPDATED_FEATURE_LIST_CONTENT], label=x_train['content'])
    dtest = lgb.Dataset(data=data[UPDATED_FEATURE_LIST_CONTENT], label=data['content'])
        
    model = lgb.train(best_params_content,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=True)])
        
    predictions = model.predict(data[UPDATED_FEATURE_LIST_CONTENT])
    full_df.loc[full_df['prompt_id'] == prompt_id, 'pred_lgb_content'] = predictions
    model.save_model(f'../lgb_models/ensemble_via_lgb3/lgb_model_content_{prompt_id}.bst')

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[348]	valid's rmse: 0.381035
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[3186]	valid's rmse: 0.441636
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[1550]	valid's rmse: 0.45561
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[166]	valid's rmse: 0.398746


In [98]:
for prompt_id, data in full_df.groupby('prompt_id'):
    x_train = full_df[full_df['prompt_id']!=prompt_id].copy()
    dtrain = lgb.Dataset(data=x_train[UPDATED_FEATURE_LIST_WORDING], label=x_train['wording'])
    dtest = lgb.Dataset(data=data[UPDATED_FEATURE_LIST_WORDING], label=data['wording'])
        
    model = lgb.train(best_params_wording,
                      num_boost_round=10000,
                      valid_names=['valid'],
                      train_set=dtrain,
                      valid_sets=dtest,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=200, verbose=True)])
        
    predictions = model.predict(data[UPDATED_FEATURE_LIST_WORDING])
    full_df.loc[full_df['prompt_id'] == prompt_id, 'pred_lgb_wording'] = predictions
    model.save_model(f'../lgb_models/ensemble_via_lgb3/lgb_model_wording_{prompt_id}.bst')

Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[86]	valid's rmse: 0.509359
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[229]	valid's rmse: 0.646193
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[388]	valid's rmse: 0.606827
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[78]	valid's rmse: 0.49196


In [99]:
import sys

In [100]:
sys.path.append('..')

In [101]:
from cles.metrics.metric import mcrmse

In [102]:
mcrmse(full_df[['content', 'wording']].values, full_df[['pred_lgb_content', 'pred_lgb_wording']].values)

(0.48866246684822034, [0.4154742412317312, 0.5618506924647094])